### rackcli layout - h4x VCV Rack layout management for patches

VCV Rack stores patches in JSON files, that's the good news. The not so good news is they're stored in the order they were added to the patch with ordinal cross references between modules and wires. This exercise is all about codifying some basic layout management services to facilitate patch construction and maintenance. As Rack matures, many of these capabilities will surely be added.

This exercise assumes your patch has plugin module width data, either from the `File.Save/SaveAs` in my 
[fork](https://github.com/dirkleas/Rack) or clicking `patch` button on 
[DLwigglz r4xH4x](https://github.com/dirkleas/DLwigglz) and using `patch.json` instead of `_layout.vcv` -- you don't need a 
full `catalog.json` for this since we're not dynamically adding plugin modules to the patch, only reorgaizing them.

In [1]:
from typing import *
import json

It'd be handy to have a grid-centric model of the patch to facilitate traversal of the patch as it sits in the rack.

In [2]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack'
patch = json.load(open(rackdir + '/_layout.vcv')) # copy patch.vcv to _layout.vcv or use catalog['width']

# layout: {row: [col]} where instance is (col, ordinal), each patch module has pos(col, row)
def hp(pixels: int) -> int:
    'convert pixels to hp'
    return(pixels / 15)

def layout(patch: Dict) -> Dict[int, List]:
    'generate ordered layout model for patch'
    layout = {}
    for i,m in enumerate(patch['modules']): layout.setdefault(m['pos'][1], []).append([m['pos'][0], i, m['width']])
    return({k:sorted(v, key=lambda x: x[0]) for (k,v) in layout.items()})

pl = layout(patch)
print(pl, patch['width'])

def align_left(layout: Dict[int, List], rows: List, start: int=0) -> Dict[int, List]:
    'left-align row modules on specified rows with optional 0-offset starting module, use slicing once available'
    for r in rows:
        sum = 0 if start == 0 else layout[r][start-1][2] + (15*layout[r][start-1][0])
        for m in layout[r][start:]:
            m[0] = int(hp(sum))
            sum += m[2]
    return(layout)

def align_right(layout: Dict[int, List], rows: List, max_width, start: int=0) -> Dict[int, List]:
    'left-align row modules on specified rows with optional 0-offset starting module, use slicing once available'
    for r in rows:
        sum = max_width - 15 # allow 1HP for scrollbar
        for m in layout[r][:None if start==0 else start-1:-1]:
            m[0] = int(hp(sum)) - int(hp(m[2]))
            sum -= m[2]
    return(layout)

pl = align_left(pl, [0,])
pl = align_right(pl, [1,], patch['width'])
# pl = align_left(pl, [0,], 1) # shift from 1th item on
# pl = align_right(pl, [1,], patch['width'], 1)
print(pl, patch['width'])

def apply_layout(layout: Dict, patch: Dict) -> Dict:
    for r in layout.values():
        for c in r: patch['modules'][c[1]]['pos'][0] = c[0]
    return(patch)

json.dump(apply_layout(pl, patch), open(rackdir + '/_layout2.vcv', 'w'), indent=2, ensure_ascii=False)

{0: [[3, 1, 270], [21, 2, 90], [27, 0, 150]], 1: [[16, 5, 150], [71, 3, 150], [81, 4, 120], [89, 6, 90]]} 1440
{0: [[0, 1, 270], [18, 2, 90], [24, 0, 150]], 1: [[61, 5, 150], [71, 3, 150], [81, 4, 120], [89, 6, 90]]} 1440
